In [ ]:
pip install -U "transformers==4.44.2" "peft==0.18.1" "accelerate==1.12.0" sentencepiece

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

DARJA2MSA_MODEL = "Saidtaoussi/AraT5_Darija_to_MSA"
ARAT5_BASE      = "UBC-NLP/AraT5v2-base-1024"
ARAT5_ADAPTER   = "HassnaaElshafei/arat5v2_adapter"

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Darija -> MSA
tok_d = AutoTokenizer.from_pretrained(DARJA2MSA_MODEL)
model_d = AutoModelForSeq2SeqLM.from_pretrained(DARJA2MSA_MODEL).to(device).eval()

# 2) MSA -> Egyptian (AraT5 base + your adapter from HF)
tok_e  = AutoTokenizer.from_pretrained(ARAT5_BASE)
base_e = AutoModelForSeq2SeqLM.from_pretrained(ARAT5_BASE).to(device)
model_e = PeftModel.from_pretrained(base_e, ARAT5_ADAPTER).to(device).eval()

def arat5_darija_to_egyptian(text, max_length=128, num_beams=5):
    # Darija -> MSA
    in1 = tok_d(text, return_tensors="pt").to(device)
    msa_ids = model_d.generate(**in1, max_length=max_length)
    msa = tok_d.decode(msa_ids[0], skip_special_tokens=True).strip()

    # MSA -> Egyptian (your fine-tuned adapter expects your prompt)
    in2 = tok_e("حول للمصري: " + msa, return_tensors="pt").to(device)
    egy_ids = model_e.generate(**in2, max_length=max_length, num_beams=num_beams)
    egy = tok_e.decode(egy_ids[0], skip_special_tokens=True).strip()
    return egy


In [8]:
moroccan_sentences = [
    "بغيت نمشي للدار دابا.",
    "هاد التليفون ديالي ماشي ديالك.",
    "توحشتك بزاف يا خويا.",
    "غادي نسافر لفرانسا السيمانة الجاية.",
    "ما بغيتش ناكل، شبعت.",
    "واش كاين شي جديد؟",
    "فين غادي تمشي ف العشية؟",
    "شحال داير هاد السروال؟",
    "عطيني شي كاس ديال الماء عفاك.",
    "الدراري كيلعبو الكورة ف الزنقة."
]

print("--- 🇲🇦 Darija -> 🇪🇬 Egyptian Using Finetuned NLLB ---")
for sent in moroccan_sentences:
    print(f"\nInput:  {sent}")
    # Use your function
    output = arat5_darija_to_egyptian(sent)
    print(f"Output: {output}")

--- 🇲🇦 Darija -> 🇪🇬 Egyptian Using Finetuned NLLB ---

Input:  بغيت نمشي للدار دابا.
Output: عايز أروح البيت دلوقتي.

Input:  هاد التليفون ديالي ماشي ديالك.
Output: ده تليفوني مش تليفونك.

Input:  توحشتك بزاف يا خويا.
Output: أنا فاكرك كتير.

Input:  غادي نسافر لفرانسا السيمانة الجاية.
Output: حأروح فرنسا الأسبوع الجاي.

Input:  ما بغيتش ناكل، شبعت.
Output: أنا مش عايز آكل، أنا شبعت.

Input:  واش كاين شي جديد؟
Output: فيه حاجة جديدة؟

Input:  فين غادي تمشي ف العشية؟
Output: إنت رايح فين بالليل؟

Input:  شحال داير هاد السروال؟
Output: القميص ده بكام؟

Input:  عطيني شي كاس ديال الماء عفاك.
Output: اديني كوبايه ميه لعفاك.

Input:  الدراري كيلعبو الكورة ف الزنقة.
Output: الأطفال بيلعبوا كرة قدم في الشارع.
